In [ ]:
import torch
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils.collators import collate_multitrack_paired
from fx_model.apply_effects_multitrack_utils import multitrack_batched_processing
import os
from datasets.tency_mastering_multitrack_paired import TencyMastering_Test
import omegaconf

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

#from fx_model.fxnormaug_v3_noiseless import FxNormAug
#randomizer=FxNormAug(sample_rate=44100, mode="train", device=device )


from fx_model.distribution_presets.uniform_RMSnorm import get_distributions_uniform
from fx_model.fx_pipeline import EffectRandomizer

distribution_uniform = get_distributions_uniform(sample_rate=44100)
effect_randomizer_uniform=EffectRandomizer(sample_rate=44100, distributions_dict=distribution_uniform, device=device)

normalize_params=omegaconf.OmegaConf.create(
    {
    "normalize_mode": "rms_dry",
    "rms_dry": -25.0
    }
)

dataset_val= TencyMastering_Test(
  mode= "dry-wet",
  segment_length= 525312,
  fs= 44100,
  stereo= True,
  tracks= "all",
  num_tracks= 1,
  path_csv= "/data5/eloi/TencyMastering/PANNs_country_pop/val_split.csv",
  normalize_params=normalize_params,
  num_examples= -1, #use all examples
  RMS_threshold_dB= -40.0,
  only_dry=True,
  random_order=True,  #randomize the order of the samples
  seed= 42
)


import torch
batch_size=32
val_loader = torch.utils.data.DataLoader(dataset=dataset_val, batch_size=batch_size, num_workers=1, collate_fn=lambda x: x)


outputs_dry = {}
outputs_wet = {}
for i,  data in enumerate(val_loader):

    collated_data = collate_multitrack_paired(data)

    x=collated_data['x'].to(device)  # x is a tensor of shape [B, N, C, L] where B is the batch size, N is the number of tracks, C is the number of channels and L is the length of the audio
    taxonomy=collated_data['taxonomies']  # taxonomy is a list of lists of taxonomies, each list is a track, each taxonomy is a string of 2 digits
    masks=collated_data['masks'].to(device)  # masks is a tensor of shape [B, N] where B is the batch size and N is the number of tracks, it is used to mask the tracks that are not present in the batch
    paths=collated_data['paths']  # paths is a list of paths to the audio files, each path is a string


    func = lambda x, taxonomy: effect_randomizer_uniform.forward(x) 

    y= multitrack_batched_processing(x.clone(), taxonomy=taxonomy, function=func, class_dependent=False, masks=masks)

    outputs_dry[i] = x.cpu()
    outputs_wet[i] = y.cpu()




-1 num_examples


0it [00:00, ?it/s]

In [ ]:
import IPython.display as ipd

ipd.Audio(outputs_dry[0][0][0].numpy(), rate=44100)
#outputs_dry[0].shape
#outputs_wet[0].shape

KeyboardInterrupt: 